In [121]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
df = pd.read_csv('precleaning_flight_df.csv')
df=df.drop(columns='Unnamed: 0')
df=df.drop(columns='index')
df=df.drop(columns='Flight time')
df['Date']=df['Date'].str.split('-').str[1].astype(int)*100+df['Date'].str.split('-').str[2].astype(int)
df.head()

,Date,Day,Airline,Departure time,Departure airport,Arrival time,Arrival airport,Price
0,601,목,ANA항공,19:55:00,GMP,22:15:00,HND,329700
1,601,목,일본항공,07:50:00,GMP,09:55:00,HND,331100
2,601,목,일본항공,19:20:00,GMP,21:30:00,HND,331100
3,601,목,아시아나항공,15:30:00,GMP,17:35:00,HND,334300
4,601,목,아시아나항공,19:50:00,GMP,22:00:00,HND,334300


In [122]:
#데이터 인코딩
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

df_encoded= df
df_encoded['Date']=(df_encoded['Date'])//100+((df_encoded['Date']).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
df_encoded['Arrival time']=df_encoded['Arrival time'].str.split(':').str[0].astype(int)
df_encoded['Departure time']=df_encoded['Departure time'].str.split(':').str[0].astype(int)

df_ohe=enc.fit_transform(df_encoded.drop(columns='Price')).toarray()


In [123]:
#테스트 트레인 데이터 분할
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target = 'Price'
X_train, X_test, y_train, y_test  = train_test_split(
    df_ohe, df_encoded['Price'], test_size=0.2, random_state=1
)

In [124]:
#LR
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(mean_squared_error(y_pred, y_test, squared = False))
print(r2_score(y_test, y_pred))


203947.569251613
0.658572707837572


In [125]:
import pickle

#파라미터 저장
pkl_filename = "flight.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr, file)

In [126]:
df = pd.read_csv('precleaning_flight_df.csv')
df=df.drop(columns='Unnamed: 0')
df=df.drop(columns='index')
df=df.drop(columns='Flight time')
df['Date']=df['Date'].str.split('-').str[1].astype(int)*100+df['Date'].str.split('-').str[2].astype(int)

#파라미터 불러오기
with open("flight.pkl", 'rb') as file:
    flightpredicter = pickle.load(file)

In [127]:
#OLS

from statsmodels.formula.api import ols
from statsmodels.regression.linear_model import OLS
olsmodel = OLS(y_train, X_train)
fittedmodel = olsmodel.fit()
y_pred = fittedmodel.predict(X_test)
print(r2_score(y_pred, y_test))



0.4954309900767334


In [84]:
#if Random
random_data=df.sample(1)
actual_price=random_data['Price'].iloc[0]
random_data=random_data.drop(columns='Price')
input_data=[list(random_data.iloc[0])]

#input_data=[['601', '목', 'ANA', '19:55', 'GMP', '22:15', 'HND']]

print(input_data)

#입력 데이터 전처리
X=pd.DataFrame(input_data, columns=['Date','Day','Airline','Departure time','Departure airport','Arrival time','Arrival airport'])
X['Date']=X['Date'].astype(int)
X['Date']=(X['Date'])//100+((X['Date']).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
X['Arrival time']=X['Arrival time'].str.split(':').str[0].astype(int)
X['Departure time']=X['Departure time'].str.split(':').str[0].astype(int)
X_encoded=enc.transform(X).toarray()

#예측
result = flightpredicter.predict(X_encoded)
print('Actual Price:', actual_price, 'Predicted Price:', result)
result = fittedmodel.predict(X_encoded)
print('Actual Price:', actual_price, 'OLS_Predicted Price:', result)

[[609, '금', '에어부산', '07:55:00', 'ICN', '10:25:00', 'NRT']]
Actual Price: 206300 Predicted Price: [230400.]
Actual Price: 206300 OLS_Predicted Price: [238720.]


In [128]:
def date_to_day(date):
    dayofweek={7:'토',1:'일',2:'월',3:'화',4:'수',5:'목',6:'금'}
    #date에서 week, 요일 추출
    m=date//100
    d=date-m*100
    if m==6: 
        mkey=4
    if m==7: 
        mkey=6
    w=(d+mkey-1)//7
    d=mkey+d-7*w
    return dayofweek[d]

kor_depart=['GMP', 'ICN']
jpn_depart=['HND', 'NRT']
cand_arriv={'GMP':'HND', 'HND':'GMP', 'ICN':'NRT', 'NRT':'ICN'}

In [161]:
#날짜, 요일, 시간만 입력하면 자동으로 추천해주는 코드 ml ver


#input_data=[['601', '목', 'ANA', '19:55', 'GMP', '22:15', 'HND']]
input_date=601
input_airline=0 #'ANA'
input_departure_airport='KOR' #'GMP'
input_departure_time_from=19
input_departure_time_to=21


X=pd.DataFrame(columns=['Date','Day','Airline','Departure time','Departure airport','Arrival time','Arrival airport'])

if input_airline==0:
    cand_airline=list(df['Airline'].unique())
else:
    cand_airline=[input_airline]

if input_departure_airport=='KOR':
    cand_airport=kor_depart
elif input_departure_airport=='JPN':
    cand_airport=jpn_depart    
else:
    cand_airport=[input_departure_airport]

cand_time=[x for x in range(input_departure_time_from,input_departure_time_to+1)]

for x in cand_airline:
    for y in cand_airport:
        for z in cand_time:
            input_data = {'Date':[input_date],'Day':[date_to_day(input_date)],'Airline':[x],'Departure time':[z],'Departure airport':[y],'Arrival time':[z+2],'Arrival airport':[cand_arriv[y]]}
            tmp=pd.DataFrame(input_data)
            X = pd.concat([X,tmp], ignore_index=True)


#입력 데이터 전처리
X['Date']=X['Date'].astype(int)
X_encoded=X
X_encoded['Date']=(X['Date'])//100+((X['Date']).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
#X['Arrival time']=X['Arrival time'].str.split(':').str[0].astype(int)
#X['Departure time']=X['Departure time'].str.split(':').str[0].astype(int)
X_encoded=enc.transform(X_encoded).toarray()

#예측
X['Price_lr']= flightpredicter.predict(X_encoded) 
X['Price_OLS'] = fittedmodel.predict(X_encoded)
X



,Date,Day,Airline,Departure time,Departure airport,Arrival time,Arrival airport,Price_lr,Price_OLS
0,6.0,목,ANA항공,19,GMP,21,HND,773120.0,781568.0
1,6.0,목,ANA항공,20,GMP,22,HND,705536.0,696576.0
2,6.0,목,ANA항공,21,GMP,23,HND,588800.0,568064.0
3,6.0,목,ANA항공,19,ICN,21,NRT,846848.0,847904.0
4,6.0,목,ANA항공,20,ICN,22,NRT,776192.0,762816.0
...,...,...,...,...,...,...,...,...,...
73,6.0,목,에어프레미아,20,GMP,22,HND,21504.0,-10496.0
74,6.0,목,에어프레미아,21,GMP,23,HND,-95232.0,-139008.0
75,6.0,목,에어프레미아,19,ICN,21,NRT,162816.0,140960.0
76,6.0,목,에어프레미아,20,ICN,22,NRT,92160.0,55808.0


In [158]:
#날짜, 요일, 시간만 입력하면 자동으로 추천해주는 코드 search ver


#input_data=[['601', '목', 'ANA', '19:55', 'GMP', '22:15', 'HND']]
input_date=601
input_airline=0 #'ANA'
input_departure_airport='KOR' #'GMP'
input_departure_time_from=19
input_departure_time_to=21


df_searchresult=df[df['Date']==input_date]
if input_airline!=0:
    df_searchresult=df_searchresult[df_searchresult['Airline']==input_airline]

if input_departure_airport=='KOR':
    cand_airport=kor_depart
elif input_departure_airport=='JPN':
    cand_airport=jpn_depart    
else:
    cand_airport=[input_departure_airport]

df_searchresult=df_searchresult[df_searchresult['Departure airport'].isin(cand_airport)]

cand_time=[x for x in range(input_departure_time_from,input_departure_time_to+1)]

df_searchresult=df_searchresult[df_searchresult['Departure time'].str.split(':').str[0].astype(int).isin(cand_time)]

df_searchresult.sort_values(by=['Price'], axis=0)

,Date,Day,Airline,Departure time,Departure airport,Arrival time,Arrival airport,Price
0,601,목,ANA항공,19:55:00,GMP,22:15:00,HND,329700
2,601,목,일본항공,19:20:00,GMP,21:30:00,HND,331100
4,601,목,아시아나항공,19:50:00,GMP,22:00:00,HND,334300
13,601,목,대한항공,19:20:00,GMP,21:30:00,HND,401800
19,601,목,ANA항공,19:50:00,GMP,22:00:00,HND,412800
22,601,목,아시아나항공,19:55:00,GMP,22:15:00,HND,503200


In [94]:
#Experiment
df_exp=df_encoded
#df_exp=df_exp.drop(columns='Arrival time')
df_exp=df_exp.drop(columns='Departure time')

####
enc2 = OneHotEncoder()

df_ohe2=enc2.fit_transform(df_exp.drop(columns='Price')).toarray()

#####
target = 'Price'
X_train2, X_test2, y_train2, y_test2  = train_test_split(
    df_ohe2, df_encoded['Price'], test_size=0.2, random_state=1
)

print(X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape)

lr2 = LinearRegression()
lr2.fit(X_train2, y_train2)

y_pred2 = lr2.predict(X_test2)

print(r2_score(y_test2, y_pred2))

(5552, 54) (1389, 54) (5552,) (1389,)
0.6290069344496634


In [ ]:
#Arrival 빼고: 0.6378200012708573
#Departure 빼고: 0.6290069344496634